In [1]:
import pandas as pd

# Read data from files
train = pd.read_csv("../word2vec-nlp-tutorial/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../word2vec-nlp-tutorial/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("../word2vec-nlp-tutorial/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

#Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews" \
      "and %d unlabeled reviews\n" % (train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviewsand 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False, remove_number=False):
    # Function to convert a document to a sequence of words, optionally removing stop words & numbers.
    # Return a list of words
    
    #1. remove HTML
    review_text = BeautifulSoup(review).get_text()
    
    #2. Remove non-letters
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    
    #3. Covert words to lower case and split them
    words = review_text.lower().split()
    
    #4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    #5. Return a list of words
    return(words)
    
    

In [3]:
# # download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download

# load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# define a func to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # Func to split a review into parsed sentences
    # Returns a list of sentences, where each sentence is a list of words
    
    #1. use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    #2. loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # if a sentence is empty, skip it
        # Otherwise, call review_to_wordlist() to get a list of words
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
        
        #Return the list of sentences(for a review, each sentence is a list of words. So this returns a list of lists)
        return sentences

In [4]:
#Now we can apply this function to prepare our data for input to Word2Vec
sentences = []
print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentence from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentence from unlabeled set


In [5]:
# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))
print(sentences[0])

75000
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [6]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [7]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-02-23 19:50:14,787 : INFO : collecting all words and their counts
2020-02-23 19:50:14,788 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-02-23 19:50:14,849 : INFO : PROGRESS: at sentence #10000, processed 250673 words, keeping 18555 word types
2020-02-23 19:50:14,916 : INFO : PROGRESS: at sentence #20000, processed 496174 words, keeping 25579 word types
2020-02-23 19:50:14,984 : INFO : PROGRESS: at sentence #30000, processed 748321 words, keeping 31383 word types


Training model...


2020-02-23 19:50:15,050 : INFO : PROGRESS: at sentence #40000, processed 998433 words, keeping 36409 word types
2020-02-23 19:50:15,120 : INFO : PROGRESS: at sentence #50000, processed 1252655 words, keeping 40516 word types
2020-02-23 19:50:15,188 : INFO : PROGRESS: at sentence #60000, processed 1503721 words, keeping 43975 word types
2020-02-23 19:50:15,251 : INFO : PROGRESS: at sentence #70000, processed 1755428 words, keeping 47041 word types
2020-02-23 19:50:15,282 : INFO : collected 48555 word types from a corpus of 1882813 raw words and 75000 sentences
2020-02-23 19:50:15,283 : INFO : Loading a fresh vocabulary
2020-02-23 19:50:15,317 : INFO : effective_min_count=40 retains 3564 unique words (7% of original 48555, drops 44991)
2020-02-23 19:50:15,318 : INFO : effective_min_count=40 leaves 1669156 word corpus (88% of original 1882813, drops 213657)
2020-02-23 19:50:15,332 : INFO : deleting the raw counts dictionary of 48555 items
2020-02-23 19:50:15,336 : INFO : sample=0.001 down

In [9]:
#The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:
model.doesnt_match("man woman child kitchen".split())
# model.doesnt_match("france england germany berlin".split())

/Users/ysy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  
2020-02-23 19:50:51,624 : WARNING : vectors for words {'kitchen'} are not present in the model, ignoring these words


'child'

In [10]:
# use the "most_similar" function to get insight into the model's word clusters
model.most_similar("man")

/Users/ysy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('doctor', 0.7730792760848999),
 ('priest', 0.7584525942802429),
 ('woman', 0.7580400705337524),
 ('scientist', 0.7402098774909973),
 ('cop', 0.7392619848251343),
 ('boy', 0.7265116572380066),
 ('himself', 0.717841386795044),
 ('girl', 0.7125262022018433),
 ('murderer', 0.7052661180496216),
 ('victim', 0.7049213647842407)]

In [11]:
model.most_similar("queen")

/Users/ysy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mary', 0.8550252914428711),
 ('joan', 0.8492039442062378),
 ('murderer', 0.8447268009185791),
 ('ruth', 0.8387153148651123),
 ('rose', 0.8385372161865234),
 ('singer', 0.8359367847442627),
 ('lady', 0.8326665163040161),
 ('former', 0.8248870372772217),
 ('boss', 0.823523223400116),
 ('mrs', 0.8226689696311951)]